<a href="https://colab.research.google.com/github/sarangbishal/Ballot-Paper-Counting-System/blob/master/Balloot_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import all required libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt

import imutils
import cv2

from collections import Counter
from imutils.perspective import four_point_transform
from imutils import contours

### Read Image and detect edges using cv2 Canny algorithm

In [0]:
FILE_PATH = "Kathmandured.jpg"

In [0]:
# Read Image
image = cv2.imread(FILE_PATH)
# Since matplotlib uses different color space
# Convert to matplotlib space for debugging purpose in notebook
image_matplotlib_compat = imutils.opencv2matplotlib(image)

# Preprocess the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# edged = cv2.Canny(blurred, 75, 200)
# auto_edged = imutils.auto_canny(gray)

# plt.imsave("image_1.jpg", edged)
# plt.imshow(auto_edged)

### Add contour detection and isolate candidate boxes

In [4]:
thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# plt.imshow(thresh)
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# List to grab each boxes contours
rows = []

# Find Most common contour area
most_common_area = Counter([cv2.contourArea(c) for c in cnts if len(cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)) == 4]).most_common(1)[0][0]
# Consider contour area upto this threshold
area_delta  = 999


i = 0
# print(most_common_area)
for c in cnts:
    
    # Calculate Perimeter of Contour
    peri = cv2.arcLength(c, True)
    # Approximate a closed polygon with minimum number of points
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    # Calculate area of current contour
    current_contour_area = cv2.contourArea(c)
    # print(current_contour_area)
    
    # If 4 corners are found and contour area satisfy minimum threshold
    if len(approx) == 4  and abs(current_contour_area - most_common_area) <= area_delta:
        rows.append(c)
    
print(len(rows))

39


### Arrange each boxes in the order as they appear on paper

In [5]:
# First sort contours in top to bottom manner
rows = contours.sort_contours(rows, method="top-to-bottom")[0]

k = 0

# List to store sorted boxes as they appear on paper
sorted_boxes = []

# For every 3 columns sort them according to left to right
# and add to sorted boxes
for (q, i) in enumerate(np.arange(0, len(rows), 3)):
    cnts = contours.sort_contours(rows[i:i + 3], method = "left-to-right")[0]
    sorted_boxes.extend(cnts)

print(len(sorted_boxes))

# List to store isolated box image converted to HSV color space
# for color detection and masking
isolated_box_img = list()

#
for c in sorted_boxes:
    k += 1
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    
    isolated_box_matplotlib_compat = four_point_transform(image_matplotlib_compat, approx.reshape(4, 2))
    isolated_box_org = four_point_transform(image, approx.reshape(4, 2))
    
    isolated_box_hsv = cv2.cvtColor(isolated_box_org, cv2.COLOR_BGR2HSV)
    
    isolated_box_img.append(isolated_box_hsv)
    # plt.imsave("naya/test_" + str(k) + ".jpg", isolated_box)

    
# plt.imsave("ou.jpg", image_matplotlib_compat)

39


### Detect red colors inside any of the boxes

In [6]:
lower_red = np.array([0,120,70])
upper_red = np.array([10,255,255])

for i, hsv in enumerate(isolated_box_img):
    mask = cv2.inRange(hsv, lower_red, upper_red)
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # print(cv2.countNonZero(mask))
  
    if(np.any(mask != 0)):
        #No color found
        print("color found at index", i)
       
    # bgless = cv2.bitwise_and(image_matplotlib_compat,image_matplotlib_compat, mask= mask)
    # plt.imshow(bgless)   
    
    

color found at index 0
color found at index 1
